# TP3 - Codage convolutionnel et l’algorithme de Viterb

L’objectif de ce travail est d’implémenter un codeur convolutionnel (2,1,3) et un dé-codeur de Viterbi.

(n, k, m) :
- n = Nombre de bits de sortie (par itération).
- k = Nombre de bits d'entrée (par itération)
- m =  Nombre d'états mémoire (profondeur du registre à décalage).

In [ ]:
!pip install mermaid-python

$s1(n) = u(n - 1)$ <br>
$s2(n) = s1(n - 1)$ <br>
$s3(n) = s2(n - 1)$ <br>

*Schéma 1:*<br>
$v_1 (n) = s1(n) + s2(n)$<br>
$v_2 (n) = s1(n) + s2(n) + s3(n)$


**Schema 1**<br>
| Input u | État Présent S1 S2 S3 | État Suivant S1 S2 S3 | Sorties v1 v2 |
|:-------------:|:------------------------------:|:------------------------------:|:----------------------------:|
| 0 | 000 | 000 | 00 |
| 1 | 000 | 100 | 00 |
| 0 | 100 | 010 | 11 |
| 1 | 100 | 110 | 11 |
| 0 | 010 | 001 | 11 |
| 1 | 010 | 101 | 11 |
| 0 | 110 | 011 | 00 |
| 1 | 110 | 111 | 00 |
| 0 | 001 | 000 | 01 |
| 1 | 001 | 100 | 01 |
| 0 | 101 | 010 | 10 |
| 1 | 101 | 110 | 10 |
| 0 | 011 | 001 | 10 |
| 1 | 011 | 101 | 10 |
| 0 | 111 | 011 | 01 |
| 1 | 111 | 111 | 01 |



In [3]:
from mermaid import Mermaid
schema_example_graph = """
stateDiagram-v2
    S1S2S3 --> S1S2S3: u = entrée/ v1v2 = sorties
"""
print("Etat avec les entrées et sorties")
Mermaid(schema_example_graph)

Etat avec les entrées et sorties


In [4]:


# Générer le graphe en Mermaid pour le schéma 1
schema_1_graph = """
stateDiagram-v2
    [*] --> 000
    000 --> 000: 0 / 00
    000 --> 100: 1 / 00
    100 --> 010: 0 / 11
    100 --> 110: 1 / 11
    010 --> 001: 0 / 11
    010 --> 101: 1 / 11
    110 --> 011: 0 / 00
    110 --> 111: 1 / 00
    001 --> 000: 0 / 01
    001 --> 100: 1 / 01
    101 --> 010: 0 / 10
    101 --> 110: 1 / 10
    011 --> 001: 0 / 10
    011 --> 101: 1 / 10
    111 --> 011: 0 / 01
    111 --> 111: 1 / 01
"""

# Utilisation de la bibliothèque Mermaid
Mermaid(schema_1_graph)



$s1(n) = u(n - 1)$ <br>
$s2(n) = s1(n - 1)$ <br>
$s3(n) = s2(n - 1)$ <br>

*Schéma 2:*<br>
$v_1 (n) = s1(n) + s3(n)$<br>
$v_2 (n) = s1(n) + s2(n) + s3(n)$


**Schema 2**<br>

| Input u | État Présent S1 S2 S3 | État Suivant S1 S2 S3 | Sorties v1 v2 |
|:-------------:|:------------------------------:|:------------------------------:|:----------------------------:|
| 0 | 000 | 000 | 00 |
| 1 | 000 | 100 | 00 |
| 0 | 100 | 010 | 11 |
| 1 | 100 | 110 | 11 |
| 0 | 010 | 001 | 01 |
| 1 | 010 | 101 | 01 |
| 0 | 110 | 011 | 10 |
| 1 | 110 | 111 | 10 |
| 0 | 001 | 000 | 11 |
| 1 | 001 | 100 | 11 |
| 0 | 101 | 010 | 00 |
| 1 | 101 | 110 | 00 |
| 0 | 011 | 001 | 10 |
| 1 | 011 | 101 | 10 |
| 0 | 111 | 011 | 01 |
| 1 | 111 | 111 | 01 |

In [14]:


schema_2_graph = """
stateDiagram-v2
    [*] --> 000
    000 --> 000: 0 / 00
    000 --> 100: 1 / 00
    100 --> 010: 0 / 11
    100 --> 110: 1 / 11
    010 --> 001: 0 / 01
    010 --> 101: 1 / 01
    110 --> 011: 0 / 10
    110 --> 111: 1 / 10
    001 --> 000: 0 / 11
    001 --> 100: 1 / 11
    101 --> 010: 0 / 00
    101 --> 110: 1 / 00
    011 --> 001: 0 / 10
    011 --> 101: 1 / 10
    111 --> 011: 0 / 01
    111 --> 111: 1 / 01
"""

# Utilisation de la bibliothèque Mermaid
Mermaid(schema_2_graph)


In [7]:
from collections import deque


N = 2
K = 1
M = 3

'''
$s1(n) = u(n - 1)$ <br>
$s2(n) = s1(n - 1)$ <br>
$s3(n) = s2(n - 1)$ <br>

*Schéma 1:*<br>
$v_1 (n) = s1(n) + s2(n)$<br>
$v_2 (n) = s1(n) + s2(n) + s3(n)$
'''

# Permet juste de choisir le schéma et de calculer les sorties
def compute_outputs(state, schema_num):
    if schema_num == 1:
        v1 = state[0] ^ state[1]
    elif schema_num == 2:
        v1 = state[0] ^ state[2]
    else:
        raise ValueError("Schéma non reconnu")
    v2 = state[0] ^ state[1] ^ state[2]
    return v1, v2

# Fonction pour générer les sorties pour une séquence donnée
def schema(sequence, schema_num):
    M = 3 
    current_state = deque([0] * M)  
    result = []
    
    for bit in sequence:
        v1, v2 = compute_outputs(current_state, schema_num)  
         # Ajout de la sortie au résultat
        result.append(f"{v1}{v2}") 
        
        current_state.pop() 
        current_state.appendleft(bit) 
        
    for _ in range(M):  
        v1, v2 = compute_outputs(current_state, schema_num)  
        result.append(f"{v1}{v2}")

        current_state.pop()
        current_state.appendleft(0)
        
    return ' '.join(result)


# Initialisation de la liste pour stocker les séquences
sequences = []

# Génération des séquences binaires de longueur 4
for i in range(16):  
    # Convertit i en binaire et enlève '0b'
    binary_string = bin(i)[2:]  
     # Ajoute des zéros à gauche
    padded_string = binary_string.zfill(4) 
    # Convertir chaque caractère en entier
    binary_sequence = [int(bit) for bit in padded_string]  
    sequences.append(binary_sequence)  

for i in range(2):
    print(f"Schéma {i + 1}")
    print("--------")
    for j,sequence in enumerate(sequences):
        print(f"Sequence {j}: {sequence} -> {schema(sequence, i + 1)}")
    print("")
    

Schéma 1
--------
Sequence 0: [0, 0, 0, 0] -> 00 00 00 00 00 00 00
Sequence 1: [0, 0, 0, 1] -> 00 00 00 00 11 11 01
Sequence 2: [0, 0, 1, 0] -> 00 00 00 11 11 01 00
Sequence 3: [0, 0, 1, 1] -> 00 00 00 11 00 10 01
Sequence 4: [0, 1, 0, 0] -> 00 00 11 11 01 00 00
Sequence 5: [0, 1, 0, 1] -> 00 00 11 11 10 11 01
Sequence 6: [0, 1, 1, 0] -> 00 00 11 00 10 01 00
Sequence 7: [0, 1, 1, 1] -> 00 00 11 00 01 10 01
Sequence 8: [1, 0, 0, 0] -> 00 11 11 01 00 00 00
Sequence 9: [1, 0, 0, 1] -> 00 11 11 01 11 11 01
Sequence 10: [1, 0, 1, 0] -> 00 11 11 10 11 01 00
Sequence 11: [1, 0, 1, 1] -> 00 11 11 10 00 10 01
Sequence 12: [1, 1, 0, 0] -> 00 11 00 10 01 00 00
Sequence 13: [1, 1, 0, 1] -> 00 11 00 10 10 11 01
Sequence 14: [1, 1, 1, 0] -> 00 11 00 01 10 01 00
Sequence 15: [1, 1, 1, 1] -> 00 11 00 01 01 10 01

Schéma 2
--------
Sequence 0: [0, 0, 0, 0] -> 00 00 00 00 00 00 00
Sequence 1: [0, 0, 0, 1] -> 00 00 00 00 11 01 11
Sequence 2: [0, 0, 1, 0] -> 00 00 00 11 01 11 00
Sequence 3: [0, 0, 1, 1] -

## 2.4 Verification avec commpy


-> je n'ai pas réussis a installer commpy sur mon environnement. J'ai réussis a installer  scikit-commpy mais quand je l importe il me dit que le module n'existe pas et parail pour la librairie Trellis `https://pypi.org/project/Trellis/`. Je ne comprends pas pourquoi...

# 3. Décodeur de Viterbi

## 3.1 trellis

## schema 1
<img src="img/schema_1.png" width="700" height="700">


## schema 2
<img src="img/schema_2.png" width="700" height="700">

In [13]:
def viterbi(encoded_sequence, schema_num):

    num_bits = len(encoded_sequence) // N
    num_states = 2 ** M 

    # Initialisation des coûts et des chemins
    cost_table = [[float('inf')] * num_states for _ in range(num_bits + 1)]
    # L'état initial a un coût de 0
    cost_table[0][0] = 0  
    predecessor_table = [[None] * num_states for _ in range(num_bits + 1)]

    # Parcours de chaque bit dans la séquence encodée
    for t in range(num_bits):
        encoded_bits = encoded_sequence[2 * t:2 * t + 2]

        for current_state in range(num_states):
            # Seulement si cet état est accessible
            if cost_table[t][current_state] < float('inf'):  
                # Test avec les bits d'entrée possibles
                for input_bit in [0, 1]:  
                    # Détermine l'état suivant
                    next_state = ((current_state << 1) | input_bit) & 0b111 
                    # Décodage de l'état en binaire
                    state_bits = [int(x) for x in f"{current_state:03b}"] 
                    
                    # Calcul des sorties pour cet état et ce bit d'entrée
                    outputs = compute_outputs(state_bits, schema_num)
                    hamming_dist = sum(o != e for o, e in zip(outputs, encoded_bits))
                    
                    # Mise à jour du coût et du chemin
                    new_cost = cost_table[t][current_state] + hamming_dist
                    if new_cost < cost_table[t + 1][next_state]:
                        cost_table[t + 1][next_state] = new_cost
                        predecessor_table[t + 1][next_state] = (current_state, input_bit)

    # Récupération du chemin optimal
    # il faut choisir l'état final avec le plus petit coût
    final_state = min(range(num_states), key=lambda s: cost_table[num_bits][s])
    decoded_bits = []
    current_state = final_state
    for t in range(num_bits, 0, -1):
        current_state, input_bit = predecessor_table[t][current_state]
        decoded_bits.insert(0, input_bit)

    return decoded_bits


# Génération des séquences d'entrée (16 séquences de longueur 4)
sequences = []
for i in range(16):  
    binary_string = bin(i)[2:].zfill(4)
    seq = [int(bit) for bit in binary_string]
    sequences.append(seq)

# Fonction pour convertir la sortie du schema en liste de bits
def convert_schema_output_to_bits(schema_output):
    # schema_output est une string du type "00 01 10 ..."
    pairs = schema_output.split()
    bits = []
    for p in pairs:
        bits.extend([int(x) for x in p])
    return bits

# Test pour le code du chema 1
print("Résultats pour le code du schema_num = 1 :")
print("------------------------------------------")
for seq in sequences:
    encoded_str = schema(seq, schema_num=1)
    encoded_bits = convert_schema_output_to_bits(encoded_str)
    decoded = viterbi(encoded_bits, schema_num=1)
    decoded = decoded[:len(seq)]
    print(f"Entrée : {seq} -> encodé : {encoded_str} -> décodé : {decoded}")

print("\nRésultats pour le code du schema_num = 2 :")
print("------------------------------------------")
for seq in sequences:
    encoded_str = schema(seq, schema_num=2)
    encoded_bits = convert_schema_output_to_bits(encoded_str)
    decoded = viterbi(encoded_bits, schema_num=2)
    decoded = decoded[:len(seq)]
    print(f"Entrée : {seq} -> encodé : {encoded_str} -> décodé : {decoded}")


Résultats pour le code du schema_num = 1 :
------------------------------------------
Entrée : [0, 0, 0, 0] -> encodé : 00 00 00 00 00 00 00 -> décodé : [0, 0, 0, 0]
Entrée : [0, 0, 0, 1] -> encodé : 00 00 00 00 11 11 01 -> décodé : [0, 0, 1, 0]
Entrée : [0, 0, 1, 0] -> encodé : 00 00 00 11 11 01 00 -> décodé : [0, 1, 0, 0]
Entrée : [0, 0, 1, 1] -> encodé : 00 00 00 11 00 10 01 -> décodé : [0, 1, 0, 1]
Entrée : [0, 1, 0, 0] -> encodé : 00 00 11 11 01 00 00 -> décodé : [1, 0, 0, 0]
Entrée : [0, 1, 0, 1] -> encodé : 00 00 11 11 10 11 01 -> décodé : [0, 1, 0, 1]
Entrée : [0, 1, 1, 0] -> encodé : 00 00 11 00 10 01 00 -> décodé : [1, 0, 1, 1]
Entrée : [0, 1, 1, 1] -> encodé : 00 00 11 00 01 10 01 -> décodé : [0, 0, 0, 1]
Entrée : [1, 0, 0, 0] -> encodé : 00 11 11 01 00 00 00 -> décodé : [1, 0, 0, 0]
Entrée : [1, 0, 0, 1] -> encodé : 00 11 11 01 11 11 01 -> décodé : [1, 0, 1, 0]
Entrée : [1, 0, 1, 0] -> encodé : 00 11 11 10 11 01 00 -> décodé : [1, 0, 1, 0]
Entrée : [1, 0, 1, 1] -> encodé : 